In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['D1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]
    Src['Datetime']                     = Query['datetime']
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
BINS = [100,200,500,800,1000,1200,1500,1800,2000,2200,2500,2800,3000,3200,3500,3800,4000,4200,4500,4800,5000,5500,6000,6500,7000,7500]

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()

    for b in BINS:  Calc[f'Chg Abs > {b}'] = (Calc['Chg Abs'] > b)
    for b in BINS:  Calc[f'Hilo > {b}']    = (Calc['Hilo']    > b)

    for b in BINS:  Calc[f'Chg Abs > {b} +1'] = Calc[f'Chg Abs > {b}'].shift(-1)
    for b in BINS:  Calc[f'Chg Abs > {b} +2'] = Calc[f'Chg Abs > {b}'].shift(-2)
    for b in BINS:  Calc[f'Chg Abs > {b} +3'] = Calc[f'Chg Abs > {b}'].shift(-3)

    for b in BINS:  Calc[f'Hilo > {b} +1'] = Calc[f'Hilo > {b}'].shift(-1)
    for b in BINS:  Calc[f'Hilo > {b} +2'] = Calc[f'Hilo > {b}'].shift(-2)
    for b in BINS:  Calc[f'Hilo > {b} +3'] = Calc[f'Hilo > {b}'].shift(-3)

    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

In [ ]:
def STATS(Calc, Col):
    pipe = []
    for b in BINS:
        Df = Calc[Calc[f'{Col} > {b}']==1]
        pipe.append({
            f'{Col} > D0':  b,
            f'{Col} > D+1': np.round(Df[f'{Col} > {b} +1'].mean()*100, 1),
            f'{Col} > D+2': np.round(Df[f'{Col} > {b} +2'].mean()*100, 1),
            f'{Col} > D+3': np.round(Df[f'{Col} > {b} +3'].mean()*100, 1),
        })
    return pd.DataFrame(pipe)

In [26]:
STATS(Calc, Col='Hilo')

,Hilo > D0,Hilo > D+1,Hilo > D+2,Hilo > D+3
0,100,100.0,100.0,100.0
1,200,100.0,100.0,100.0
2,500,100.0,100.0,100.0
3,800,99.1,99.0,99.0
4,1000,94.9,94.9,95.2
5,1200,86.9,89.0,87.9
6,1500,72.8,74.5,74.7
7,1800,55.9,58.8,58.3
8,2000,50.2,50.8,51.0
9,2200,43.7,45.2,43.9


In [27]:
STATS(Calc, Col='Chg Abs')

,Chg Abs > D0,Chg Abs > D+1,Chg Abs > D+2,Chg Abs > D+3
0,100,93.2,93.1,93.6
1,200,88.1,87.5,87.9
2,500,68.7,71.1,71.3
3,800,49.2,54.7,53.6
4,1000,40.7,42.6,45.9
5,1200,31.2,33.4,34.9
6,1500,23.7,23.0,22.7
7,1800,18.4,14.9,15.9
8,2000,17.6,9.5,8.8
9,2200,16.0,6.7,8.4
